### Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
from pathlib import Path
scripts_path = Path("../Data-Preprocessing/").resolve()
sys.path.append(str(scripts_path))
import os
os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'

In [17]:
import pickle
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestCentroid
from scripts.data_visualiser import *
from scripts.data_loader import *
from scripts.data_preprocessor import *
from scripts.temporal_data_preprocessor import *
from scripts.temporal_data_loader import *
from scripts.temporal_visualiser import *
from scripts.temporal_chanel_refinement import *
from model_scripts.get_statistics import *
from model_scripts.dataset_creation import *
from model_scripts.train_model_ae import *
from model_scripts.model_visualiser import *
from model_scripts.superpixel import *
from Pipeline.pre_processing_pipeline import *
from Pipeline.temporal_preprocessing_pipeline import *
from model_scripts.pre_trained_extraction import *
import numpy as np
import preprocessing_config as config
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA
import skimage.measure
import torch
import torch.nn as nn
import torch.optim as optim

### Loading the data

In [4]:
# temp_pipeline = PreProcessingPipelineTemporal()

In [5]:
# train_fn, dataloader_train = temp_pipeline.get_processed_trainloader(64, 'indexbands', vi_type='msi')

Train

In [6]:
temporal_images = load_field_images_temporal(config.base_directory_temporal_train)
border_removed_images_train = blacken_field_borders_temporal(temporal_images)
field_numbers_train, indices_images_train = allbands_temporal_cubes(border_removed_images_train)

len(indices_images_train), indices_images_train[0][0].shape

(2425, (64, 64, 10))

Test

In [7]:
temporal_images_test = load_field_images_temporal(config.base_directory_temporal_test)
border_removed_images = blacken_field_borders_temporal(temporal_images_test)
field_numbers_test, indices_images_test = allbands_temporal_cubes(border_removed_images)

(len(indices_images_test), indices_images_test[0][0].shape)

(48, (64, 64, 10))

In [8]:
image_tensor_train = np.stack(indices_images_train)  # Shape: (N x 7 x 64 x 64 x 6)
image_tensor_test = np.stack(indices_images_test)   # Shape: (N x 7 x 64 x 64 x 6)

image_tensor_train.shape, image_tensor_test.shape

((2425, 7, 64, 64, 10), (48, 7, 64, 64, 10))

### Super-patches

In [9]:
image_tensor_train = torch.tensor(image_tensor_train, dtype=torch.float32).permute(0, 1, 4, 2, 3)  # (N, T, H, W, C) -> (N, T, C, H, W)
image_tensor_test = torch.tensor(image_tensor_test, dtype=torch.float32).permute(0, 1, 4, 2, 3)  # (N, T, H, W, C) -> (N, T, C, H, W)
image_tensor_train.shape, image_tensor_test.shape

(torch.Size([2425, 7, 10, 64, 64]), torch.Size([48, 7, 10, 64, 64]))

In [10]:
train_patches, train_patch_coordinates = non_overlapping_sliding_window(image_tensor_train, field_numbers_train, patch_size=4)
test_patches, test_patch_coordinates = non_overlapping_sliding_window(image_tensor_test, field_numbers_test, patch_size=4)

In [11]:
len(test_patches), test_patches[0].shape

(1197, torch.Size([7, 10, 4, 4]))

In [12]:
# visualize_temporal_stack_rgb(border_removed_images[0])

In [ ]:
# visualize_single_patch_temporal_rgb(test_patches[2], test_patch_coordinates[2])

In [ ]:
test_patch_coordinates[0]

In [ ]:
visualize_patches(image_tensor_test, field_numbers_test, test_patch_coordinates, 0, patch_size=4)

### Data Loaders - for Autoencoders

In [ ]:
train_patches = torch.stack(train_patches)
test_patches = torch.stack(test_patches)
test_patches.shape

In [ ]:
batch_size = 64
dataloader_train = create_data_loader(train_patches, train_patch_coordinates, batch_size=batch_size, shuffle=True)

for batch_inputs, batch_field_numbers in dataloader_train:
    print("Batch Inputs Shape:", batch_inputs.shape) 
    print("Batch Field Numbers:", batch_field_numbers)
    break  # Show one batch

In [ ]:
batch_size = 64
dataloader_test = create_data_loader(test_patches, test_patch_coordinates, batch_size=batch_size, shuffle=False)

for batch_inputs, batch_field_numbers in dataloader_test:
    print("Batch Inputs Shape:", batch_inputs.shape) 
    print("Batch Field Numbers:", batch_field_numbers)
    break  # Show one batch

## Modeling: k-means

Updated threshold thing --- Check again!! Resume from here!!!

In [ ]:
train_patches_tensor = torch.stack(train_patches)  # Convert to tensor [N, T, C, H, W]
test_patches_tensor = torch.stack(test_patches)

In [ ]:
kmeans = train_kmeans_patches(train_patches_tensor, n_clusters=2, random_state=64)

train_patch_predictions = kmeans.predict(train_patches_tensor.reshape(train_patches_tensor.size(0), -1).numpy())
test_patch_predictions = kmeans.predict(test_patches_tensor.reshape(test_patches_tensor.size(0), -1).numpy())

# Assign field labels
threshold = 10
train_field_labels = assign_field_labels(train_patch_coordinates, train_patch_predictions, threshold)
test_field_labels = assign_field_labels(test_patch_coordinates, test_patch_predictions, threshold)

In [ ]:
accuracy, report = evaluate_test_labels(test_field_labels, config.labels_path)
print(f"Test Accuracy: {accuracy}")
print(report)

In [ ]:
# # Save train predictions to Excel
# save_train_predictions_to_excel(train_field_labels, "train_predictions.xlsx")

## Pre-trained Models + k-means

In [14]:
train_patches_tensor = torch.stack(train_patches) 
test_patches_tensor = torch.stack(test_patches)
train_patches_tensor.shape, test_patches_tensor.shape

(torch.Size([54614, 7, 10, 4, 4]), torch.Size([1197, 7, 10, 4, 4]))

In [18]:
model = SpatioTemporalFeatureExtractor(
    spatial_weights=ResNet18_Weights.SENTINEL2_ALL_MOCO,
    input_channels=10,
)

train_features = model(train_patches_tensor)
print(train_features.shape)  


IndexError: index 7 is out of bounds for dimension 1 with size 7

: 